In [1]:
import os 
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd                      # 가장 기초적인 데이터전처리 패키지
import re                                # 정규표현식을 사용할 수 있는 패키지
from datetime import datetime  
import urllib.request
from PIL import Image
from pytesseract import *

In [62]:
data = pd.DataFrame({"date" : [],
                    "image" : [],
                    "name" : [],
                    "price" : [],
                    "link" : [],
                    "size":[]})

In [2]:
base_url = 'http://dabagirl.co.kr'

In [3]:
req = requests.get(base_url)
if req == 200 :
    soup = BeautifulSoup(req.content, 'html.parser')
else :
    req = requests.get(base_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup = BeautifulSoup(req.content, 'html.parser')

In [4]:
req

<Response [200]>

In [5]:
category_1 = soup.find('ul',{'class' : 'xans-element- xans-layout xans-layout-category'})

In [6]:
category_2 = category_1.find_all('li',{'class' : 'd1 xans-record-'})

In [5]:
category_2

[<li class="d1 xans-record-" df-cate-depth="1" df-cate-no="516"><a href="/product/list.html?cate_no=516">당/일/배/송</a></li>,
 <li class="d1 xans-record-" df-cate-depth="1" df-cate-no="329"><a href="/product/list.html?cate_no=329">NEW~10%</a></li>,
 <li class="d1 xans-record-" df-cate-depth="1" df-cate-no="67"><a href="/product/list.html?cate_no=67">BEST 100</a></li>,
 <li class="d1 xans-record-" df-cate-depth="1" df-cate-no="903"><a href="/product/list.html?cate_no=903">OFFICE</a></li>,
 <li class="d1 xans-record-" df-cate-depth="1" df-cate-no="26"><a href="/product/list.html?cate_no=26">OUTER</a></li>,
 <li class="d1 xans-record-" df-cate-depth="1" df-cate-no="25"><a href="/product/list.html?cate_no=25">TOP</a></li>,
 <li class="d1 xans-record-" df-cate-depth="1" df-cate-no="28"><a href="/product/list.html?cate_no=28">DRESS</a></li>,
 <li class="d1 xans-record-" df-cate-depth="1" df-cate-no="130"><a href="/product/list.html?cate_no=130">PANTS</a></li>,
 <li class="d1 xans-record-" df-ca

In [6]:
category_list_name=[]
for i in range(len(category_2)):
    category_list_nm = category_2[i].find('a').text
    category_list_name.append(category_list_nm)

In [7]:
category_list_name= category_list_name[4:8]

In [8]:
category_url =[]
for i in range(len(category_2)):
    category_list = base_url + category_2[i].find('a')['href']
    category_url.append(category_list)

In [9]:
category_url

['http://dabagirl.co.kr/product/list.html?cate_no=516',
 'http://dabagirl.co.kr/product/list.html?cate_no=329',
 'http://dabagirl.co.kr/product/list.html?cate_no=67',
 'http://dabagirl.co.kr/product/list.html?cate_no=903',
 'http://dabagirl.co.kr/product/list.html?cate_no=26',
 'http://dabagirl.co.kr/product/list.html?cate_no=25',
 'http://dabagirl.co.kr/product/list.html?cate_no=28',
 'http://dabagirl.co.kr/product/list.html?cate_no=130',
 'http://dabagirl.co.kr/product/list.html?cate_no=129',
 'http://dabagirl.co.kr/product/list.html?cate_no=38',
 'http://dabagirl.co.kr/product/list.html?cate_no=850',
 'http://dabagirl.co.kr/product/list.html?cate_no=852',
 'http://dabagirl.co.kr/product/list.html?cate_no=64',
 'http://dabagirl.co.kr/product/list.html?cate_no=889']

In [10]:
category_url2 = category_url[4:8]

In [11]:
category_url2

['http://dabagirl.co.kr/product/list.html?cate_no=26',
 'http://dabagirl.co.kr/product/list.html?cate_no=25',
 'http://dabagirl.co.kr/product/list.html?cate_no=28',
 'http://dabagirl.co.kr/product/list.html?cate_no=130']

In [12]:
start_url=category_url2[0]
page_url = category_url2[0] + '&page=' + '{}'

In [13]:
req2 = requests.get(start_url)
if req2 == 200 :
    soup2 = BeautifulSoup(req2.content, 'html.parser')
else :
    req2 = requests.get(start_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup2 = BeautifulSoup(req2.content, 'html.parser')


In [14]:
page1_a = soup2.find('div',{'class':'xans-element- xans-product xans-product-normalpaging df-base-paging'})
page2_a = page1_a.find_all('li',{'class','xans-record-'})

In [15]:
page1_b = page1_a.find_all('a')[-1]['href'] #가장 마지막 페이지로 가는 url
page2_b = start_url + page1_b

#page2_a[-1:] #리스트형식 그대로 뽑아주기때문에 태그추출이안됨
#page2_a[-1] #요소를 뽑아주기 때문에 태그추출 가능

In [16]:
if start_url+page2_a[-1].find('a')['href'] != page2_b:
    req3 = requests.get(page2_b)
    if req3 == 200 :
        soup3 = BeautifulSoup(req3.content, 'html.parser')
    else :
        req3 = requests.get(page2_b, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
        soup3 = BeautifulSoup(req3.content, 'html.parser')

    page3_b = soup3.find('div',{'class':'xans-element- xans-product xans-product-normalpaging df-base-paging'})
    page4_b = page3_b.find_all('li',{'class','xans-record-'})

    page5 = len(page2_a) + len(page4_b)
else:
    page5=len(page2_a)

    


In [17]:
page5

3

In [18]:
for i in range(1,page5+1):
    req4 = requests.get(page_url.format(str(i)))
    if req4 == 200 :
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    else :
        req4 = requests.get(page_url.format(str(i)), headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    
    products = soup4.find_all('li', {'class' : 'item xans-record-'})

In [59]:
for product in products:
    try:
        link = product.find('div', {'class' : 'thumbnail'}).find('a')['href']
        link_url = base_url + link

        name = product.find('p',{'class':'name'}).find_all('span')[1].find('b').text
        name = re.sub('[^가-힣0-9a-zA-Z\\s\+\[\]\,\.\=\-\_\~\!\@\#\$\%\^\&\*\(\)\;\:\?]','',name)

        price_0 = product.find('ul', {'class' : 'xans-element- xans-product xans-product-listitem'}) # 가격정보가 있는 곳의 클래스
        price_1 = price_0.find('li', {'class' : 'xans-record-'}) # 원 가격, 할인가격 등 다양한 가격이 있는데, 할인가격으로 표시해주는 것
        price_2 = price_1.find_all('span')[1].text

        req5 = requests.get(link_url)
        if req5 == 200 :
            soup5 = BeautifulSoup(req5.content, 'html.parser')
        else :
            req5 = requests.get(link_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
            soup5 = BeautifulSoup(req5.content, 'html.parser')

        image = soup5.find('div',{'class':'thumbnail'})
        image_url = image.find('img')['src']


        src=[]
        size = soup5.find('div',{'class':'cont'})
        size_img = size.find_all('img')

        for i in range(len(size_img)) :
            img_src = size_img[i]['src'] # 이미지의 주소에 대한 정보는 img 태그 내의 src 속성에 있다. 이를 가지고 오는 것.
            src.append(img_src)

        for result in src :
                if "size" in result :
                    x = result
                else:
                    pass

        insert_data = pd.DataFrame({"date" : datetime.today().strftime("%Y-%m-%d"),
                                "image" : [image_url],
                                "name" : [name],
                                "price" : [price_2],
                                "link" : [link_url],
                                "size" : [x]})

        data = data.append(insert_data)
    except:
        pass

data = data.drop_duplicates(subset='image', keep='last') # 중복된 부분이 있다면 가장 최근것으로 대체
data.index = range(len(data))


 

In [63]:
price_2

'17,900원'

In [20]:
link_url

'http://dabagirl.co.kr/product/detail.html?product_no=32062&cate_no=26&display_group=1'

In [21]:
image_url

'//dabagirl.co.kr/web/product/big/201509/32062_shop1_113904.jpg'

In [22]:
size_img

[<img src="//dabagirl.cdn.smart-img.com/dfitmujizipup/0.jpg"/>,
 <img src="//dabagirl.cdn.smart-img.com/dfitmujizipup/1.jpg"/>,
 <img src="//dabagirl.cdn.smart-img.com/dfitmujizipup/11.jpg"/>,
 <img src="//dabagirl.cdn.smart-img.com/dfitmujizipup/111.jpg"/>,
 <img src="//dabagirl.cdn.smart-img.com/dfitmujizipup/2.jpg"/>,
 <img src="//dabagirl.cdn.smart-img.com/dfitmujizipup/ss.jpg"/>,
 <img src="//dabagirl.cdn.smart-img.com/dfitmujizipup/size.jpg"/>,
 <img src="//dabagirl.cdn.smart-img.com/qoo10/return_notice.jpg"/>,
 <img src="//dabagirl.cdn.smart-img.com/notice/a6.jpg"/>,
 <img src="//dabagirl.cdn.smart-img.com/notice/notice.jpg"/>,
 <img src="//dabagirl.cdn.smart-img.com/notice/barcode01.jpg"/>]

In [23]:
src

['//dabagirl.cdn.smart-img.com/dfitmujizipup/0.jpg',
 '//dabagirl.cdn.smart-img.com/dfitmujizipup/1.jpg',
 '//dabagirl.cdn.smart-img.com/dfitmujizipup/11.jpg',
 '//dabagirl.cdn.smart-img.com/dfitmujizipup/111.jpg',
 '//dabagirl.cdn.smart-img.com/dfitmujizipup/2.jpg',
 '//dabagirl.cdn.smart-img.com/dfitmujizipup/ss.jpg',
 '//dabagirl.cdn.smart-img.com/dfitmujizipup/size.jpg',
 '//dabagirl.cdn.smart-img.com/qoo10/return_notice.jpg',
 '//dabagirl.cdn.smart-img.com/notice/a6.jpg',
 '//dabagirl.cdn.smart-img.com/notice/notice.jpg',
 '//dabagirl.cdn.smart-img.com/notice/barcode01.jpg']

In [24]:
x

'//dabagirl.cdn.smart-img.com/dfitmujizipup/size.jpg'

In [ ]:
try :
    pred_data = pd.read_csv('다바걸.csv', encoding='cp949') # 이전에 있던 파일을 포함해 조정하는 것
    all_data = pred_data.append(data)
    data = all_data.drop_duplicates(subset='image', keep='last')
    data.index = range(len(data))
except :
    pass


# 다바걸 아우터

In [65]:
data = pd.DataFrame({"date" : [],
                    "image" : [],
                    "name" : [],
                    "price" : [],
                    "link" : [],
                    "size":[]})
base_url = 'http://dabagirl.co.kr'

req = requests.get(base_url)
if req == 200 :
    soup = BeautifulSoup(req.content, 'html.parser')
else :
    req = requests.get(base_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup = BeautifulSoup(req.content, 'html.parser')

category_1 = soup.find('ul',{'class' : 'xans-element- xans-layout xans-layout-category'})
category_2 = category_1.find_all('li',{'class' : 'd1 xans-record-'})

category_url =[]
for i in range(len(category_2)):
    category_list = base_url + category_2[i].find('a')['href']
    category_url.append(category_list)
    
category_url2 = category_url[4:8]    

start_url=category_url2[0]
page_url = category_url2[0] + '&page=' + '{}'    
    
req2 = requests.get(start_url)
if req2 == 200 :
    soup2 = BeautifulSoup(req2.content, 'html.parser')
else :
    req2 = requests.get(start_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup2 = BeautifulSoup(req2.content, 'html.parser')
page1_a = soup2.find('div',{'class':'xans-element- xans-product xans-product-normalpaging df-base-paging'})
page2_a = page1_a.find_all('li',{'class','xans-record-'})
    
page1_b = page1_a.find_all('a')[-1]['href'] #가장 마지막 페이지로 가는 url
page2_b = start_url + page1_b

#page2_a[-1:] #리스트형식 그대로 뽑아주기때문에 태그추출이안됨
#page2_a[-1] #요소를 뽑아주기 때문에 태그추출 가능    
    
if start_url+page2_a[-1].find('a')['href'] != page2_b:
    req3 = requests.get(page2_b)
    if req3 == 200 :
        soup3 = BeautifulSoup(req3.content, 'html.parser')
    else :
        req3 = requests.get(page2_b, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
        soup3 = BeautifulSoup(req3.content, 'html.parser')

    page3_b = soup3.find('div',{'class':'xans-element- xans-product xans-product-normalpaging df-base-paging'})
    page4_b = page3_b.find_all('li',{'class','xans-record-'})

    page5 = len(page2_a) + len(page4_b)
else:
    page5=len(page2_a)    
    
for i in range(1,page5+1):
    req4 = requests.get(page_url.format(str(i)))
    if req4 == 200 :
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    else :
        req4 = requests.get(page_url.format(str(i)), headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    
    products = soup4.find_all('li', {'class' : 'item xans-record-'})
    
    
    for product in products:
        try:
            link = product.find('div', {'class' : 'thumbnail'}).find('a')['href']
            link_url = base_url + link

            name = product.find('p',{'class':'name'}).find_all('span')[1].find('b').text
            name = re.sub('[^가-힣0-9a-zA-Z\\s\+\[\]\,\.\=\-\_\~\!\@\#\$\%\^\&\*\(\)\;\:\?]','',name)

            price_0 = product.find('ul', {'class' : 'xans-element- xans-product xans-product-listitem'}) # 가격정보가 있는 곳의 클래스
            price_1 = price_0.find('li', {'class' : 'xans-record-'}) # 원 가격, 할인가격 등 다양한 가격이 있는데, 할인가격으로 표시해주는 것
            price_2 = price_1.find_all('span')[1].text

            req5 = requests.get(link_url)
            if req5 == 200 :
                soup5 = BeautifulSoup(req5.content, 'html.parser')
            else :
                req5 = requests.get(link_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                    'Chrome/71.0.3578.98 Safari/537.36')})
                soup5 = BeautifulSoup(req5.content, 'html.parser')

            image = soup5.find('div',{'class':'thumbnail'})
            image_url = image.find('img')['src']


            src=[]
            size = soup5.find('div',{'class':'cont'})
            size_img = size.find_all('img')

            for i in range(len(size_img)) :
                img_src = size_img[i]['src'] # 이미지의 주소에 대한 정보는 img 태그 내의 src 속성에 있다. 이를 가지고 오는 것.
                src.append(img_src)

            for result in src :
                    if "size" in result :
                        x = result
                    else:
                        pass

            insert_data = pd.DataFrame({"date" : datetime.today().strftime("%Y-%m-%d"),
                                    "image" : [image_url],
                                    "name" : [name],
                                    "price" : [price_2],
                                    "link" : [link_url],
                                    "size" : [x]})

            data = data.append(insert_data)
        except:
            pass

data = data.drop_duplicates(subset='image', keep='last') # 중복된 부분이 있다면 가장 최근것으로 대체
data.index = range(len(data))
    
    
    
    
    

In [66]:
data

,date,image,name,price,link,size
0,2019-03-07,//dabagirl.co.kr/web/product/big/201903/54390c...,[여리여리]콩닥콩닥가디건,"28,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20170322...
1,2019-03-07,//dabagirl.co.kr/web/product/big/201902/09a5ac...,[핫딜]스캔큐롯자켓,"52,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190219...
2,2019-03-07,//dabagirl.co.kr/web/product/big/201902/9fcd76...,[핫딜]기본충실모던자켓,"55,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190222...
3,2019-03-07,//dabagirl.co.kr/web/product/big/201902/f36bd7...,[핫딜]마렘진주롱코트,"53,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190225...
4,2019-03-07,//dabagirl.co.kr/web/product/big/201902/be59ad...,[핫딜]레전드핏자켓,"49,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20180405...
5,2019-03-07,//dabagirl.co.kr/web/product/big/201903/bbdfee...,[핫딜]타카롱코트,"84,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190305...
6,2019-03-07,//dabagirl.co.kr/web/product/big/201903/41c09c...,[핫딜]로우플언발자켓_B1OU+로우플슬랙스_B1PA_SE(10%할인),"64,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190306...
7,2019-03-07,//dabagirl.co.kr/web/product/big/201903/fb7892...,카펠모던가디건,"17,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190306...
8,2019-03-07,//dabagirl.co.kr/web/product/big/201903/f345a8...,리브미벨트가디건,"23,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20180323...
9,2019-03-07,//dabagirl.co.kr/web/product/big/201903/d6b329...,기다려가디건세트,"24,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20180330...


In [ ]:
# 각 카테고리별로 세트로 판매하는 상품이 존재하여 사이즈표도 두개가 생기며 혼란스럽기때문에 세트판매상품은 예외처리

In [75]:
import urllib.request # 이미지 파일을 저장하기 위한 패키지, 웹사이트의 이미지 주소가 있으면 다운 받을 수 있다.

In [79]:
for i in range(0,10) : # 사이즈 표에 대한 이미지 파일을 저장하는 것, 반복문을 통해 하나씩 저장해주었다.
    url = 'http:' + data['size'].iloc[i]
    name = data["name"].iloc[i] + category_list_name[0] + 'size' + str(i) + '.jpg' # ex) 아우터size100.jpg 형식으로 파일을 저장
    urllib.request.urlretrieve(url, name)

# 다바걸 상의

In [84]:
data = pd.DataFrame({"date" : [],
                    "image" : [],
                    "name" : [],
                    "price" : [],
                    "link" : [],
                    "size":[]})
base_url = 'http://dabagirl.co.kr'

req = requests.get(base_url)
if req == 200 :
    soup = BeautifulSoup(req.content, 'html.parser')
else :
    req = requests.get(base_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup = BeautifulSoup(req.content, 'html.parser')

category_1 = soup.find('ul',{'class' : 'xans-element- xans-layout xans-layout-category'})
category_2 = category_1.find_all('li',{'class' : 'd1 xans-record-'})

category_url =[]
for i in range(len(category_2)):
    category_list = base_url + category_2[i].find('a')['href']
    category_url.append(category_list)
    
category_url2 = category_url[4:8]    

start_url=category_url2[1]
page_url = category_url2[1] + '&page=' + '{}'    
    
req2 = requests.get(start_url)
if req2 == 200 :
    soup2 = BeautifulSoup(req2.content, 'html.parser')
else :
    req2 = requests.get(start_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup2 = BeautifulSoup(req2.content, 'html.parser')
page1_a = soup2.find('div',{'class':'xans-element- xans-product xans-product-normalpaging df-base-paging'})
page2_a = page1_a.find_all('li',{'class','xans-record-'})
    
page1_b = page1_a.find_all('a')[-1]['href'] #가장 마지막 페이지로 가는 url
page2_b = start_url + page1_b

#page2_a[-1:] #리스트형식 그대로 뽑아주기때문에 태그추출이안됨
#page2_a[-1] #요소를 뽑아주기 때문에 태그추출 가능    
    
if start_url+page2_a[-1].find('a')['href'] != page2_b:
    req3 = requests.get(page2_b)
    if req3 == 200 :
        soup3 = BeautifulSoup(req3.content, 'html.parser')
    else :
        req3 = requests.get(page2_b, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
        soup3 = BeautifulSoup(req3.content, 'html.parser')

    page3_b = soup3.find('div',{'class':'xans-element- xans-product xans-product-normalpaging df-base-paging'})
    page4_b = page3_b.find_all('li',{'class','xans-record-'})

    page5 = len(page2_a) + len(page4_b)
else:
    page5=len(page2_a)    
    
for i in range(1,page5+1):
    req4 = requests.get(page_url.format(str(i)))
    if req4 == 200 :
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    else :
        req4 = requests.get(page_url.format(str(i)), headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    
    products = soup4.find_all('li', {'class' : 'item xans-record-'})
    
    
    for product in products:
        try:
            link = product.find('div', {'class' : 'thumbnail'}).find('a')['href']
            link_url = base_url + link

            name = product.find('p',{'class':'name'}).find_all('span')[1].find('b').text
            name = re.sub('[^가-힣0-9a-zA-Z\\s\+\[\]\,\.\=\-\_\~\!\@\#\$\%\^\&\*\(\)\;\:\?]','',name)

            price_0 = product.find('ul', {'class' : 'xans-element- xans-product xans-product-listitem'}) # 가격정보가 있는 곳의 클래스
            price_1 = price_0.find('li', {'class' : 'xans-record-'}) # 원 가격, 할인가격 등 다양한 가격이 있는데, 할인가격으로 표시해주는 것
            price_2 = price_1.find_all('span')[1].text

            req5 = requests.get(link_url)
            if req5 == 200 :
                soup5 = BeautifulSoup(req5.content, 'html.parser')
            else :
                req5 = requests.get(link_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                    'Chrome/71.0.3578.98 Safari/537.36')})
                soup5 = BeautifulSoup(req5.content, 'html.parser')

            image = soup5.find('div',{'class':'thumbnail'})
            image_url = image.find('img')['src']


            src=[]
            size = soup5.find('div',{'class':'cont'})
            size_img = size.find_all('img')

            for i in range(len(size_img)) :
                img_src = size_img[i]['src'] # 이미지의 주소에 대한 정보는 img 태그 내의 src 속성에 있다. 이를 가지고 오는 것.
                src.append(img_src)

            for result in src :
                    if "size" in result :
                        x = result
                    else:
                        pass

            insert_data = pd.DataFrame({"date" : datetime.today().strftime("%Y-%m-%d"),
                                    "image" : [image_url],
                                    "name" : [name],
                                    "price" : [price_2],
                                    "link" : [link_url],
                                    "size" : [x]})

            data = data.append(insert_data)
        except:
            pass

data = data.drop_duplicates(subset='image', keep='last') # 중복된 부분이 있다면 가장 최근것으로 대체
data.index = range(len(data))
    
    
    
    
    

In [81]:
data

,date,image,name,price,link,size
0,2019-03-07,//dabagirl.co.kr/web/product/big/201903/6f0652...,찬란한세트블랑,"26,500원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20180418...
1,2019-03-07,//dabagirl.co.kr/web/product/big/201903/b24dda...,매일입을래티셔츠,"14,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20180228...
2,2019-03-07,//dabagirl.co.kr/web/product/big/201903/b335ac...,설레이는블라우스,"33,500원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190306...
3,2019-03-07,//dabagirl.co.kr/web/product/big/201903/d6b329...,기다려가디건세트,"24,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20180330...
4,2019-03-07,//dabagirl.co.kr/web/product/big/201903/3bdb7c...,보들보들루즈셔츠,"35,700원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190306...
5,2019-03-07,//dabagirl.co.kr/web/product/big/201903/e019a6...,심쿵라인패드탑,"10,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20180322...
6,2019-03-07,//dabagirl.co.kr/web/product/big/201903/e5ace6...,카라뷔스티에나시,"23,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20180221...
7,2019-03-07,//dabagirl.co.kr/web/product/big/201903/d56a61...,서있는여자반팔탑,"22,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190306...
8,2019-03-07,//dabagirl.co.kr/web/product/big/201903/2ad2cf...,모리트임니트,"21,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190306...
9,2019-03-07,//dabagirl.co.kr/web/product/big/201903/790eda...,아이비크롭탑,"9,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190306...


In [9]:
import urllib.request # 이미지 파일을 저장하기 위한 패키지, 웹사이트의 이미지 주소가 있으면 다운 받을 수 있다.

In [83]:
for i in range(0,10) : # 사이즈 표에 대한 이미지 파일을 저장하는 것, 반복문을 통해 하나씩 저장해주었다.
    url = 'http:' + data['size'].iloc[i]
    name = category_list_name[2] + data["name"].iloc[i]  + 'size' + str(i) + '.jpg' # ex) 아우터size100.jpg 형식으로 파일을 저장
    urllib.request.urlretrieve(url, name)

**상의,아우터 는 총길이/어깨/가슴/밑단/소매기장/암홀/소매부리로 고정.**

# 다바걸 드레스

In [2]:
data = pd.DataFrame({"date" : [],
                    "image" : [],
                    "name" : [],
                    "price" : [],
                    "link" : [],
                    "size":[]})
base_url = 'http://dabagirl.co.kr'

req = requests.get(base_url)
if req == 200 :
    soup = BeautifulSoup(req.content, 'html.parser')
else :
    req = requests.get(base_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup = BeautifulSoup(req.content, 'html.parser')

category_1 = soup.find('ul',{'class' : 'xans-element- xans-layout xans-layout-category'})
category_2 = category_1.find_all('li',{'class' : 'd1 xans-record-'})

category_url =[]
for i in range(len(category_2)):
    category_list = base_url + category_2[i].find('a')['href']
    category_url.append(category_list)
    
category_url2 = category_url[4:8]    

start_url=category_url2[2]
page_url = category_url2[2] + '&page=' + '{}'    
    
req2 = requests.get(start_url)
if req2 == 200 :
    soup2 = BeautifulSoup(req2.content, 'html.parser')
else :
    req2 = requests.get(start_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup2 = BeautifulSoup(req2.content, 'html.parser')
page1_a = soup2.find('div',{'class':'xans-element- xans-product xans-product-normalpaging df-base-paging'})
page2_a = page1_a.find_all('li',{'class','xans-record-'})
    
page1_b = page1_a.find_all('a')[-1]['href'] #가장 마지막 페이지로 가는 url
page2_b = start_url + page1_b

#page2_a[-1:] #리스트형식 그대로 뽑아주기때문에 태그추출이안됨
#page2_a[-1] #요소를 뽑아주기 때문에 태그추출 가능    
    
if start_url+page2_a[-1].find('a')['href'] != page2_b:
    req3 = requests.get(page2_b)
    if req3 == 200 :
        soup3 = BeautifulSoup(req3.content, 'html.parser')
    else :
        req3 = requests.get(page2_b, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
        soup3 = BeautifulSoup(req3.content, 'html.parser')

    page3_b = soup3.find('div',{'class':'xans-element- xans-product xans-product-normalpaging df-base-paging'})
    page4_b = page3_b.find_all('li',{'class','xans-record-'})

    page5 = len(page2_a) + len(page4_b)
else:
    page5=len(page2_a)    
    
for i in range(1,page5+1):
    req4 = requests.get(page_url.format(str(i)))
    if req4 == 200 :
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    else :
        req4 = requests.get(page_url.format(str(i)), headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    
    products = soup4.find_all('li', {'class' : 'item xans-record-'})
    
    
    for product in products:
        try:
            link = product.find('div', {'class' : 'thumbnail'}).find('a')['href']
            link_url = base_url + link

            name = product.find('p',{'class':'name'}).find_all('span')[1].find('b').text
            name = re.sub('[^가-힣0-9a-zA-Z\\s\+\[\]\,\.\=\-\_\~\!\@\#\$\%\^\&\*\(\)\;\:\?]','',name)

            price_0 = product.find('ul', {'class' : 'xans-element- xans-product xans-product-listitem'}) # 가격정보가 있는 곳의 클래스
            price_1 = price_0.find('li', {'class' : 'xans-record-'}) # 원 가격, 할인가격 등 다양한 가격이 있는데, 할인가격으로 표시해주는 것
            price_2 = price_1.find_all('span')[1].text

            req5 = requests.get(link_url)
            if req5 == 200 :
                soup5 = BeautifulSoup(req5.content, 'html.parser')
            else :
                req5 = requests.get(link_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                    'Chrome/71.0.3578.98 Safari/537.36')})
                soup5 = BeautifulSoup(req5.content, 'html.parser')

            image = soup5.find('div',{'class':'thumbnail'})
            image_url = image.find('img')['src']


            src=[]
            size = soup5.find('div',{'class':'cont'})
            size_img = size.find_all('img')

            for i in range(len(size_img)) :
                img_src = size_img[i]['src'] # 이미지의 주소에 대한 정보는 img 태그 내의 src 속성에 있다. 이를 가지고 오는 것.
                src.append(img_src)

            for result in src :
                    if "size" in result :
                        x = result
                    else:
                        pass

            insert_data = pd.DataFrame({"date" : datetime.today().strftime("%Y-%m-%d"),
                                    "image" : [image_url],
                                    "name" : [name],
                                    "price" : [price_2],
                                    "link" : [link_url],
                                    "size" : [x]})

            data = data.append(insert_data)
        except:
            pass

data = data.drop_duplicates(subset='image', keep='last') # 중복된 부분이 있다면 가장 최근것으로 대체
data.index = range(len(data))
    
    
    
    
    

In [3]:
data

,date,image,name,price,link,size
0,2019-03-07,//dabagirl.co.kr/web/product/big/201902/4705a4...,[핫딜]애플트위드원피스,"74,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190222...
1,2019-03-07,//dabagirl.co.kr/web/product/big/201902/c523aa...,[핫딜]에덴롱원피스,"69,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190215...
2,2019-03-07,//dabagirl.co.kr/web/product/big/201903/80679b...,[핫딜]플레어져원피스,"53,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190304...
3,2019-03-07,//dabagirl.co.kr/web/product/big/201903/41c09c...,[핫딜]로우플언발자켓_B1OU+로우플슬랙스_B1PA_SE(10%할인),"64,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190306...
4,2019-03-07,//dabagirl.co.kr/web/product/big/201903/9c80da...,프릴진볼륨원피스,"42,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190307...
5,2019-03-07,//dabagirl.co.kr/web/product/big/201903/7f0f74...,봄을부르는원피스,"29,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190306...
6,2019-03-07,//dabagirl.co.kr/web/product/big/201903/58e42b...,릿서스펜더원피스,"37,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20180313...
7,2019-03-07,//dabagirl.co.kr/web/product/big/201903/0feae8...,리쿠세트원피스,"23,900원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20190305...
8,2019-03-07,//dabagirl.co.kr/web/product/big/201901/38705c...,특별한날에원피스,"42,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20180315...
9,2019-03-07,//dabagirl.co.kr/web/product/big/201810/edcbfe...,오스카롱원피스,"39,000원",http://dabagirl.co.kr/product/detail.html?prod...,//dabagirl.cdn.smart-img.com/dabagirl/20181008...


In [18]:
name_list=[]
for i in range(0,10) : # 사이즈 표에 대한 이미지 파일을 저장하는 것, 반복문을 통해 하나씩 저장해주었다.
    url = 'http:' + data['size'].iloc[i]
    name = category_list_name[2] + data["name"].iloc[i]  + 'size' + str(i) + '.jpg' # ex) 아우터size100.jpg 형식으로 파일을 저장
    urllib.request.urlretrieve(url, name)
    name_list.append(name)

In [19]:
name_list

['DRESS[핫딜]애플트위드원피스size0.jpg',
 'DRESS[핫딜]에덴롱원피스size1.jpg',
 'DRESS[핫딜]플레어져원피스size2.jpg',
 'DRESS[핫딜]로우플언발자켓_B1OU+로우플슬랙스_B1PA_SE(10%할인)size3.jpg',
 'DRESS프릴진볼륨원피스size4.jpg',
 'DRESS봄을부르는원피스size5.jpg',
 'DRESS릿서스펜더원피스size6.jpg',
 'DRESS리쿠세트원피스size7.jpg',
 'DRESS특별한날에원피스size8.jpg',
 'DRESS오스카롱원피스size9.jpg']

**상품마다 다른 경우도 있음**

**따라서 +나 세트라고 써져있는 것은 예외처리 해야할 듯.**

In [2]:
for i in range(0,10):
    img = Image.open(name_list[i])
    text = pytesseract.image_to_string(img,lang='kor')
    test = re.split('\n',text)
    
    for j in range(len(test)):
        if len(re.findall('.*단면0.*',test[j]))>=1:
            x = test[j]
        else:
            pass
    #else사이에 무슨문제가...
    y= test.index(x)
    z=test[y:y+3]

NameError: name 'Image' is not defined

In [22]:
z

['(단면00) 총길이 어깨     가슴    허리     힙     밑단    암홀 소매길이',
 '이 126 41      서      33 38 97      19 58',
 '(벨트기장-150)                                                                        (소매둘레-11)']

**일단 데이터형식으로 저장하기 위해서 txt형식이 아니라 csv형식으로 저장은 각 부위를 columns로 수치를 레코드로 넣고
상품별로 csv를 만드는 코드를 짜는 것이 나을듯 혹은 다른 방법이 있음 더 생각하거나 구글링, 질문**

In [ ]:
for i in range(len(test5)):
    if len(re.findall('.*총길이.*어깨.*',test5[i]))>=1:
        x4 = test5[i]
    else:
        pass
    #사이에 무슨 문제 있나?
y4= test5.index(x4)
z4=test5[y4:y4+3]
   

# csv저장

In [1]:
for i in range(0,10):
    img = Image.open(name_list[i])
    text = pytesseract.image_to_string(img,lang='kor')
    test = re.split('\n',text)
    
    for j in range(len(test)):
        if len(re.findall('.*단면0.*',test[j]))>=1:
            x = test[j]
        else:
            pass
    #else사이에 무슨문제가...
y= test.index(x)
z=test[y:y+3]

data = pd.DataFrame({"size" : []})

#word = [] #이러면 너무 복잡해지는데 
for i in range(len(re.split(' ',test7[y6]))):
    try:
        word=re.split(' ',test7[y6])[i]
    except:
        pass
    
    
    
    result = ""
    for j in re.findall('[0-9a-zA-Z가-힣]',re.split(' ',test7[y6])[i]):
        if len(j) != 0: #비어있는 리스트는 삭제하고 넣으려고 조건문을 넣었지만 흠... 밑에처럼나옴
            result +=j
        else:
            pass
    
    insert_data = pd.DataFrame({"size" : [result]})
    
    data = data.append(insert_data)
    df = data[data["size"] != ""]
    df.index= range(len(df))
    
    data2 = pd.DataFrame({"num" : []})
    
    for a in range(len(list(filter(None,re.split(' ',test7[y6+1]))))):
        try:
            word2=re.split(' ',test7[y6+1])[a]
        except:
            pass



        result2 = ""
        for b in re.findall('[0-9a-zA-Z가-힣]',list(filter(None,re.split(' ',test7[y6+1])))[a]):
            if len(b) != 0: #비어있는 리스트는 삭제하고 넣으려고 조건문을 넣었지만 흠... 밑에처럼나옴
                result2 +=b
            else:
                pass

        insert_data2 = pd.DataFrame({"num" : [result2]})

        data2 = data2.append(insert_data2)
        df2 = data2[data2["num"] != ""]
        df2.index= range(len(df2))
        
    df3 = pd.concat([df,df2],axis=1)

NameError: name 'pd' is not defined